In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point
# from pyproj import CRS
from itertools import chain
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
data_file = 'sardinia_geo.pkl'
force = False

if force or not os.path.isfile(data_file):
    powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2023 SP2\Python\3.10'
    if powerfactory_path not in sys.path:
        sys.path.append(powerfactory_path)
    import powerfactory as pf

    app = pf.GetApplication()
    if app is None:
        raise Exception('Cannot get PowerFactory application')
    print('Got PowerFactory application.')
    
    project_name = '\\Terna_Inerzia\\V2020_Rete_Sardegna_2021_06_03cr'
    err = app.ActivateProject(project_name)
    if err:
        raise Exception(f'Cannot activate project {project_name}')
    print(f'Activated project "{project_name}".')
    
    def get_coords(obj_type):
        objs = app.GetCalcRelevantObjects('*.' + obj_type)
        names, coords = [], []
        for obj in objs:
            if (obj.HasAttribute('outserv') and obj.outserv) or \
            not obj.HasAttribute('GPSlat') or not obj.HasAttribute('GPSlon'):
                continue
            lat,long = obj.GPSlat, obj.GPSlon
            if lat != 0.0 and long != 0.0:
                coords.append([lat,long])
                names.append(obj.loc_name)
        return names, np.array(coords)

    obj_types = {'sites': 'ElmSite', 'terminals': 'ElmTerm', 'substations': 'ElmSubstat'}
    names, coords = {}, {}
    for k,v in obj_types.items():
        names[k], coords[k] = get_coords(v)
    lines = [obj for obj in app.GetCalcRelevantObjects('*.ElmLne') if not obj.outserv]
    names['lines'], coords['lines'], line_ratings = [], [], []
    for line in lines:
        if len(line.GPScoords) > 0:
            names['lines'].append(line.loc_name)
            coords['lines'].append(np.array(line.GPScoords))
            line_ratings.append(line.typ_id.uline)
    coords['lines'] = np.array(coords['lines'], dtype=object)
    line_ratings = np.array(line_ratings)

    data = {
        'names': names,
        'coords': coords,
        'line_ratings': line_ratings
    }
    pickle.dump(data, open(data_file, 'wb'))
    
else:
    data = pickle.load(open(data_file, 'rb'))
    names = data['names']
    coords = data['coords']
    line_ratings = data['line_ratings']